In [9]:
import plotly.express as px
import pandas as pd
import csv
import math

In [24]:
vdem = pd.read_csv('V-Dem-CY-Full+Others-v14.csv', encoding= 'utf-8')
#bti = pd.read_csv('free-fair-elections-bti.csv', quoting=csv.QUOTE_NONE, encoding= 'utf-8')
fiw = pd.read_excel('Country_and_Territory_Ratings_and_Statuses_FIW_1973-2024.xlsx', sheet_name = 'Country Ratings, Statuses ', skiprows = 2)
fiw_years = pd.read_excel('Country_and_Territory_Ratings_and_Statuses_FIW_1973-2024.xlsx', sheet_name = 'Country Ratings, Statuses ', skiprows=1, nrows=1, header = None)
#wrp = pd.read_csv('WRP_national.csv', quoting=csv.QUOTE_NONE, encoding= 'utf-8')

/var/folders/8s/g7rdmd6n0578b6r7knmss1gr0000gn/T/ipykernel_41854/1711662958.py:1: DtypeWarning:

Columns (364,365,366,399,415,804,836,837,924,1240,1257,1486,3094,3168,3169,3341,3342,3344,3345,3347,3350,3352) have mixed types. Specify dtype option on import or set low_memory=False.



In [25]:
vdem = vdem.replace('"', '', regex=True)
new_headers = []
for header in vdem.columns: 
    header = header.strip('"') 
    new_headers.append(header)
vdem.columns = new_headers

In [26]:
PR = [col for col in fiw.columns if col.startswith('PR')]
CL = [col for col in fiw.columns if col.startswith('CL')]
Status = [col for col in fiw.columns if col.startswith('Status')]
fiw_r = pd.lreshape(fiw, {'PR':PR, 'CL':CL, 'Status':Status})

In [27]:
fiw_r

,Country,PR,CL,Status
0,Afghanistan,4,5,PF
1,Albania,7,7,NF
2,Algeria,6,6,NF
3,Andorra,4,3,PF
4,Angola,-,-,-
...,...,...,...,...
10450,"Yemen, N.",-,-,-
10451,"Yemen, S.",-,-,-
10452,Yugoslavia,-,-,-
10453,Zambia,4,4,PF


In [28]:
country_rename = {
    "Burma/Myanmar": "Myanmar (Burma)",
    "Myanmar": "Myanmar (Burma)",
    "Republic of Vietnam": "South Vietnam",
    "Vietnam, S.": "South Vietnam",
    "Vietnam, N.": "North Vietnam", # add code that says if its before a certain year, change some Vietnams to North Vietnam
    "Yemen, S.": "South Yemen",
    "Yemen, N.": "North Yemen", # add code that says if its before a certain year, change some Yemens to North Yemen
    "Republic of the Congo": "Congo (Brazzaville)",
    "Congo": "Congo (Brazzaville)",
    "Democratic Republic of the Congo ": "Congo (Kinshasa)",
    "Germany, W.": "West Germany", # add code that says if its before a certain year, change some Germanies to West Germany
    "Germany, E.": "East Germany", 
    "German Democratic Republic": "East Germany",
    "Czechia": "Czech Republic", # add code that says if its before a certain year, change it to Czechoslovakia
    "United States": "United States of America",
    "Cote d'Ivoire": "Ivory Coast",
    "Cabo Verde": "Cape Verde",
    "T√ºrkiye": "Turkey",
    "East Timor": "Timor-Leste",
    "The Gambia": "Gambia"
}

vdem["country_name"] = vdem["country_name"].replace(country_rename) 
#bti["Entity"] = bti["Entity"].replace(country_rename) 
fiw_r["Country"] = fiw_r["Country"].replace(country_rename) 


for i in range(len(vdem)):
    if (vdem.loc[i,'country_name'] == 'Vietnam') and (1945 < vdem.loc[i,'year'] <= 1976):
        vdem.loc[i,'country_name'] = 'North Vietnam'
    if (vdem.loc[i,'country_name'] == 'Yemen') and (1918 < vdem.loc[i,'year'] <= 1990):
        vdem.loc[i,'country_name'] = 'North Yemen'
    if (vdem.loc[i,'country_name'] == 'Germany') and (1949 < vdem.loc[i,'year'] <= 1990):
        vdem.loc[i,'country_name'] = 'West Germany'
    if (vdem.loc[i,'country_name'] == 'Czech Republic') and (vdem.loc[i,'year'] < 1993):
        vdem.loc[i,'country_name'] = 'Czechoslovakia'

In [29]:
vdem

,country_name,country_text_id,country_id,year,historical_date,project,historical,histname,codingstart,codingend,...,e_mipopula,e_miurbani,e_pefeliex,e_wb_pop,e_pechmor,e_miinteco,e_civil_war,e_miinterc,e_pt_coup,e_pt_coup_attempts
0,Mexico,MEX,3,1789,1789-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
1,Mexico,MEX,3,1790,1790-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
2,Mexico,MEX,3,1791,1791-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,Mexico,MEX,3,1792,1792-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
4,Mexico,MEX,3,1793,1793-12-31,1,1,Viceroyalty of New Spain,1789,2023,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27729,Piedmont-Sardinia,SPD,373,1857,1857-12-31,1,1,Kingdom of Piedmont-Sardinia,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27730,Piedmont-Sardinia,SPD,373,1858,1858-12-31,1,1,Kingdom of Piedmont-Sardinia,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27731,Piedmont-Sardinia,SPD,373,1859,1859-12-31,1,1,Kingdom of Piedmont-Sardinia [incorporates Lom...,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27732,Piedmont-Sardinia,SPD,373,1860,1860-12-31,1,1,NaN,1789,1861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Import data from GitHub
data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_with_codes.csv')

In [8]:
data

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [32]:
# Create basic choropleth map
fig = px.choropleth(vdem, locations='country_text_id', color='v2x_corr',
                    projection='natural earth', animation_frame='year',
                    title='Political Corruption Index by Country')
fig.show()